# Playing with Microsoft Graph (via REST API and APP)
(https://docs.microsoft.com/en-us/graph/api/overview)
- Requirement: the APP should have the following 'API permissions': 
 - Microsoft Graph:
   - Directory.Read.All
   - Group.Read.All
   - User.Read.All

# Required information

In [ ]:
appId = '' 
appSecret = '' 
tenantId = ''

# Required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import urllib.parse

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the token to access via the app

In [ ]:
def getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://graph.microsoft.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

# Instantiating previous function

In [ ]:
access_token = getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get list of users
https://docs.microsoft.com/en-us/graph/api/user-list?view=graph-rest-1.0&tabs=http

In [ ]:
def graph_listusers(access_token):
    url = "https://graph.microsoft.com/v1.0/users"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating previous function

In [ ]:
df_users = graph_listusers(access_token)

# Post-processing (extending) the list of users

In [ ]:
df_users['EXT#'] = df_users['userPrincipalName'].apply(lambda x: "External" if "#EXT#" in x else "Internal")

In [ ]:
df_users['domains'] = df_users['userPrincipalName'].apply(lambda x: x.split("#EXT")[0].split("_")[-1] if "#EXT#" in x else x.split('@')[1])

# How the data looks like?

In [ ]:
df_users

# How many users are related to each domain connected to the tenant?

In [ ]:
df_user_grouped = df_users.groupby(['domains','EXT#']).size().reset_index(name='counts')
df_user_grouped

ATTENTION: The internal pie plot angle is MANUALY adjusted

In [ ]:
fig, ax = plt.subplots()

size = 0.3
vals = np.array([[60., 32.], [37., 40.], [29., 10.]])

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
inner_colors = cmap(np.array([1, 2, 5, 6, 9, 10]))

wedges, texts = ax.pie(df_user_grouped['counts'],radius=1, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'))

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(str(df_user_grouped['counts'][i])+' ' + df_user_grouped['domains'][i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                horizontalalignment=horizontalalignment, **kw)

df_users['EXT#'].value_counts().plot(kind='pie', ax=ax, radius=1-size, 
                                     colors=['blue','red'],
                                     wedgeprops=dict(width=size, edgecolor='w'), 
                                     startangle=85.5,
                                     label="",
                                    legend=None)

plt.legend(title='Domain type',loc='upper left', bbox_to_anchor=(-0.3, 1, 0, 0))

ax.set_title('Distribution of users related to domains',y=1.08)
plt.show()

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function get list of created objects
https://docs.microsoft.com/en-us/graph/api/user-list-createdobjects?view=graph-rest-1.0&tabs=http

In [ ]:
def graph_listcreatedobjects(access_token, user_id):
    url = "https://graph.microsoft.com/v1.0/users/%s/createdObjects" % (user_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Enriching the list of users to with the list of created objects

In [ ]:
df_users['createdobjectids'] = df_users['id'].apply(lambda x: graph_listcreatedobjects(access_token, x) if len(graph_listcreatedobjects(access_token, x))>0 else False)

#  How the list of users looks like with the list of created objects?

In [ ]:
df_users

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get a list of groups in a tenant

In [ ]:
def graph_listgroups(access_token):
    url = "https://graph.microsoft.com/v1.0/groups"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_groups = graph_listgroups(access_token)

# What groups exist in this tenant?

In [ ]:
df_groups['displayName'] 

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the list of members in a group

In [ ]:
def graph_listmembers(access_token, group_id):
    url = "https://graph.microsoft.com/v1.0/groups/%s/members" % (group_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the list of owners in a group

In [ ]:
def graph_listowners(access_token, group_id):
    url = "https://graph.microsoft.com/v1.0/groups/%s/owners" % (group_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Enriching the list of groups with the list of members and the list of owners

In [ ]:
df_groups['members'] = df_groups['id'].apply(lambda x: graph_listmembers(access_token,x)['userPrincipalName'].values if len(graph_listmembers(access_token,x))>0 else False)

In [ ]:
df_groups['members_size'] = df_groups['members'].apply(lambda x: len(x) if x is not False else 0)

In [ ]:
df_groups['owners'] = df_groups['id'].apply(lambda x: graph_listowners(access_token,x)['userPrincipalName'].values if len(graph_listowners(access_token,x))>0 else False)

# How many members, which members, and what owners are for each group?

In [ ]:
df_groups[['displayName','members','members_size','owners']]

In [ ]:
fig, ax = plt.subplots()
df_groups[['displayName','members_size']].sort_values(by='members_size').plot(kind='barh',ax=ax)
ax.set_yticklabels(df_groups[['displayName','members_size']].sort_values(by='members_size')['displayName'], fontsize=10)

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the list of directory roles

In [ ]:
def graph_listdirectoryroles(access_token):
    url = "https://graph.microsoft.com/v1.0/directoryRoles"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_listdirectoryroles = graph_listdirectoryroles(access_token)

# How the data looks like?

In [ ]:
df_listdirectoryroles

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the list of members in each directory role

In [ ]:
def graph_listmembersdirectoryroles(access_token,directoryrole_id):
    url = "https://graph.microsoft.com/v1.0/directoryRoles/%s/members" % (directoryrole_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Enriching the list of directory roles with the members

In [ ]:
df_listdirectoryroles['members'] = df_listdirectoryroles['id'].apply(lambda x: graph_listmembersdirectoryroles(access_token,x)['userPrincipalName'].values if len(graph_listmembersdirectoryroles(access_token,x))>0 else False)

In [ ]:
df_listdirectoryroles['members_size'] = df_listdirectoryroles['members'].apply(lambda x: len(x) if x is not False else 0)

# How many people with specific roles and who are these people?

In [ ]:
df_listdirectoryroles[df_listdirectoryroles['members_size'] != 0].sort_values(by='members_size', ascending=False)